In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import fetch_20newsgroups
data=fetch_20newsgroups()
data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [2]:
train=fetch_20newsgroups(subset='train',categories=data.target_names)
test=fetch_20newsgroups(subset='test',categories=data.target_names)

In [3]:
print(len(train.data))

11314


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vector=CountVectorizer()
X_train=vector.fit_transform(train.data)
X_train.shape


(11314, 130107)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer=TfidfTransformer()
X_train_tfidf=vectorizer.fit_transform(X_train)
X_train_tfidf.shape


(11314, 130107)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

clf=GradientBoostingClassifier(n_estimators=50,verbose=2).fit(X_train_tfidf,train.target)

      Iter       Train Loss   Remaining Time 
         1           2.2026            6.38m
         2           1.9756            6.45m
         3           1.8175            6.29m
         4           1.6981            6.15m
         5           1.6015            6.05m
         6           1.5209            5.94m
         7           1.4467            5.78m
         8           1.3820            5.62m
         9           1.3284            5.47m
        10           1.2775            5.32m
        11           1.2335            5.19m
        12           1.1908            5.08m
        13           1.1541            4.95m
        14           1.1201            4.81m
        15           1.0865            4.66m
        16           1.0553            4.52m
        17           1.0270            4.38m
        18           0.9993            4.24m
        19           0.9744            4.10m
        20           0.9482            3.96m
        21           0.9265            3.82m
        2

In [7]:
X_test=vector.transform(test.data)
X_test_tfidf=vectorizer.transform(X_test)
predicted= clf.predict(X_test_tfidf)

In [10]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print("accuracy:",accuracy_score(test.target,predicted))
print("confusion matrix:",test.target,predicted)
print(classification_report(test.target,predicted,target_names=test.target_names))

accuracy: 0.7385820499203399
confusion matrix: [ 7  5  0 ...  9  6 15] [12 12  0 ...  9  3 15]
                          precision    recall  f1-score   support

             alt.atheism       0.80      0.63      0.71       319
           comp.graphics       0.67      0.66      0.67       389
 comp.os.ms-windows.misc       0.70      0.68      0.69       394
comp.sys.ibm.pc.hardware       0.62      0.72      0.67       392
   comp.sys.mac.hardware       0.79      0.78      0.79       385
          comp.windows.x       0.83      0.60      0.69       395
            misc.forsale       0.81      0.84      0.83       390
               rec.autos       0.86      0.72      0.78       396
         rec.motorcycles       0.90      0.85      0.87       398
      rec.sport.baseball       0.92      0.85      0.88       397
        rec.sport.hockey       0.93      0.86      0.89       399
               sci.crypt       0.89      0.81      0.85       396
         sci.electronics       0.29      0.68 